In [1]:
import IPython.display
import sys
from rdflib import *
from rdflib.collection import Collection
import glob
import os
import re
import random
import csv
import itertools
from SPARQLWrapper import SPARQLWrapper, JSON, BASIC

In [2]:
def getFirstAndFinalStates(activity):
    result = None
    queryString = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX : <http://example.org/virtualhome2kg/ontology/>
    PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    select distinct ?firstState ?firstStateType ?firstObject ?finalState ?finalStateType ?finalObject where {
        {
            select  (Max(?number) as ?max) where {
                <%activity> :action ?action .
                ?action :actionNumber ?number .
            }
        }
        <%activity> :action ?firstAction ;
                  :action ?finalAction .
        ?firstAction :actionNumber "0"^^xsd:int ;
                :situationBeforeAction ?firstSituation ;
                :object ?firstObject .
        ?firstState :partOf ?firstSituation ;
                    :isStateOf ?firstObject .
        optional {?firstState :state ?firstStateType .}
        ?finalAction :actionNumber ?max ;
                :situationAfterAction ?finalSituation ;
                :object ?finalObject .
        ?finalState :partOf ?finalSituation ;
                    :isStateOf ?finalObject .
        optional {?finalState :state ?finalStateType .}
    }
    """
    sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials('admin', '')
    queryString = queryString.replace("%activity", activity)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)

    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        result = bindings
    except  Exception as e:
        print(e)
    return result

In [3]:
def readRoutine(path):
    with open(path) as f:
        reader = csv.reader(f)
        list = [row for row in reader]
        return list

In [4]:
def getActivity(activity_name):
    result = None
    queryString = """
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        select distinct *
        where { 
            ?s a ho:%s
        } ORDER BY RAND() limit 1 
    """
    sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials('admin', '')
    sparql.setQuery(queryString % activity_name)
    sparql.setReturnFormat(JSON)

    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        result = bindings[0]['s']['value']
    except  Exception as e:
        print(e)
        result = e
    return result

In [5]:
def checkConsistency(activity1_value, activity2_value):
    finalStateList1 = activity1_value["finalState"]
    firstStateList2 = activity2_value["firstState"]
    executable = True
    for finalState in finalStateList1:
        finalStateObject = stateMap[finalState]["object"]
        finalStateTypes = stateMap[finalState]["state"]
        for firstState in firstStateList2:
            firstStateObject = stateMap[firstState]["object"]
            firstStateTypes = stateMap[firstState]["state"]
            if finalStateObject == firstStateObject:
                for finalStateType in finalStateTypes:
                    if finalStateType in inverse_states:
                        if inverse_states[finalStateType] in firstStateTypes:
                            #print([finalStateType, firstStateTypes])
                            #さらに、そのobjectがactivtyの対象になっているかチェックし、対象となっていなければ上書きする
                            executable = checkTargetObject(firstState)
                            assert executable
                    else:
                        #print([finalState, firstState])
                        continue
    return executable

In [6]:
def checkTargetObject(state):
    queryString = """
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        ASK { 
            <%s> :partOf ?situation ;
                      :isStateOf ?object .
            ?action (:situationBeforeAction|:situationAfterAction) ?situation .
            ?activity :action ?action ;
                      :action ?allActions .
            ?allActions ho:object ?object .
        }
    """
    sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials('admin', '')
    sparql.setQuery(queryString % state)
    sparql.setReturnFormat(JSON)

    try :
        json = sparql.query().convert()
        if json['boolean'] is True:
            result = False
        else:
            result = True
    except  Exception as e:
        print(e)
        result = e
    return result

In [7]:
def update(activity1, activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        DELETE {
            ?firstState :partOf ?situation ;
                rdf:type :State ;
                :isStateOf ?object ;
                :nextState ?nexState .
        } INSERT { 
            ?activity1 :nextActivity ?activity2 .
            ?finalState :partOf ?situation ; 
                        :nextState ?nextState .
        } WHERE {
            {
                SELECT ?activity1 (Max(?number) as ?max) WHERE {
                    BIND(<%activity1> AS ?activity1)
                    ?activity1 :action ?action .
                    ?action :actionNumber ?number .
                } group by ?activity1
            }
            ?activity1 :action ?finalAction .
            ?finalAction :actionNumber ?finalNumber ;
                    :situationAfterAction ?finalSituation .
            ?finalState :partOf ?finalSituation ;
                        :isStateOf ?object .
            filter(?finalNumber = ?max)
            BIND(<%activity2> AS ?activity2)
            ?activity2 :action ?firstAction .
            ?firstAction :actionNumber "0"^^xsd:int ;
                    :situationBeforeAction ?firstSituation .
            ?firstState :partOf ?firstSituation ;
                        :isStateOf ?object ;
                        :partOf ?situation .
            optional {?firstState :nextState ?nextState}
        }
    """
    sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented/statements")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials('admin', '')
    sparql.method = "POST"
    sparql.queryType = "INSERT"
    queryString = queryString.replace("%activity1", activity1)
    queryString = queryString.replace("%activity2", activity2)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    
    try :
        json = sparql.query().convert()
        bindings = json
        result = bindings
    except  Exception as e:
        print(e)
    return result

In [8]:
def checkNextActivityIsRoutined(activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        ASK {
            ?prevActivity :nextActivity <%activity2> .
        }
    """
    sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials('admin', '')
    queryString = queryString.replace("%activity2", activity2)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    
    try :
        json = sparql.query().convert()
        result = json["boolean"]
    except  Exception as e:
        print(e)
    return result

In [9]:
# def getFinalStatesForUpdate(activity1):
#     result = None
#     queryString = """
#     PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
#     PREFIX : <http://example.org/virtualhome2kg/ontology/>
#     PREFIX ex: <http://example.org/virtualhome2kg/instance/>
#     PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
#     SELECT ?object ?firstState ?situation ?prevState .
#     WHERE {
#         {
#             SELECT ?activity1 (Max(?number) as ?max) WHERE {
#                 BIND(<%activity1> AS ?activity1)
#                 ?activity1 :action ?action .
#                 ?action :actionNumber ?number .
#             } group by ?activity1
#         }
#         ?activity1 :action ?finalAction .
#         ?finalAction :actionNumber ?finalNumber ;
#                 :situationAfterAction ?finalSituation .
#         ?finalState :partOf ?finalSituation ;
#                     :isStateOf ?object ;
#                     :partOf ?situation .
#         optional {?prevState :nextState ?finalState}
#     }
#     """
#     try :
#         sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_augmented2/statements")
#         sparql.setHTTPAuth(BASIC)
#         sparql.setCredentials('admin', '')
#         queryString = queryString.replace("%activity1", activity1)
#         sparql.setQuery(queryString)
#         sparql.setReturnFormat(JSON)
#         json = sparql.query().convert()
#         bindings = json
#         for binding in bindings:
#             binding[]
#     except  Exception as e:
#         print(e.args)
#     return result

In [10]:
# def getFirstStatesForUpdate(activity2, object_name):
#     result = None
#     queryString = """
#     PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
#     PREFIX : <http://example.org/virtualhome2kg/ontology/>
#     PREFIX ex: <http://example.org/virtualhome2kg/instance/>
#     PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
#     SELECT *
#     WHERE {
#         ?activity2 :action ?firstAction .
#             ?firstAction :actionNumber "0"^^xsd:int;
#                 :situationBeforeAction ?firstSituation .
#             ?firstState :partOf ?firstSituation ;
#                         :isStateOf <%object_name> .
#     }
#     """
#     try :
#         sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_augmented2/statements")
#         sparql.setHTTPAuth(BASIC)
#         sparql.setCredentials('admin', '')
#         queryString = queryString.replace("%activity2", activity2)
#         queryString = queryString.replace("%object_name", object_name)
#         sparql.setQuery(queryString)
#         sparql.setReturnFormat(JSON)
#         json = sparql.query().convert()
#         bindings = json
#         result = bindings
#     except  Exception as e:
#         print(e.args)
#     return result

In [11]:
def addNextState(activity1, activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        INSERT {
            ?prevState :nextState ?finalState .
        } WHERE {
             {
                SELECT ?activity1 (Max(?number) as ?max) WHERE {
                    BIND(<%activity1> AS ?activity1)
                    ?activity1 :action ?action .
                    ?action :actionNumber ?number .
                } group by ?activity1
            }
            ?activity1 :action ?finalAction .
            ?finalAction :actionNumber ?max ;
                    :situationAfterAction ?finalSituation .
            ?finalState :partOf ?finalSituation ;
                        :isStateOf ?object1 .
            ?prevState :nextState ?finalState .

            BIND(<%activity2> AS ?activity2)
            ?activity2 :action ?firstAction .
            ?firstAction :actionNumber "0"^^xsd:int;
                :situationBeforeAction ?firstSituation .
            ?firstState :partOf ?firstSituation ;
                        :isStateOf ?object2 .
            filter(?object1 = ?object2)
        }
    """
    try :
        sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented/statements")
        sparql.setHTTPAuth(BASIC)
        sparql.setCredentials('admin', '')
        sparql.method = "POST"
        sparql.queryType = "INSERT"
        queryString = queryString.replace("%activity1", activity1)
        queryString = queryString.replace("%activity2", activity2)
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        json = sparql.query().convert()
        bindings = json
        result = bindings
    except  Exception as e:
        print(e.args)
    return result

In [12]:
def deleteFinalState(activity1, activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        DELETE {
            ?finalState ?p ?o .
        } WHERE {
             {
                SELECT (Max(?number) as ?max) WHERE {
                    <%activity1> :action ?action .
                    ?action :actionNumber ?number .
                }
            }
            <%activity1> :action ?finalAction .
            ?finalAction :actionNumber ?max ;
                    :situationAfterAction ?finalSituation .
            ?finalState :partOf ?finalSituation ;
                        :isStateOf ?object1 .
            ?finalState ?p ?o .
            <%activity2> :action ?firstAction .
            ?firstAction :actionNumber "0"^^xsd:int;
                :situationBeforeAction ?firstSituation .
            ?firstState :partOf ?firstSituation ;
                        :isStateOf ?object2 .
            filter(?object1 = ?object2)
        }
    """
    try :
        sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented/statements")
        sparql.setHTTPAuth(BASIC)
        sparql.setCredentials('admin', '')
        sparql.method = "POST"
        sparql.queryType = "INSERT"
        queryString = queryString.replace("%activity1", activity1)
        queryString = queryString.replace("%activity2", activity2)
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        json = sparql.query().convert()
        bindings = json
        result = bindings
    except  Exception as e:
        print(e.args)
    return result

In [13]:
def updateRoutine(activity1, activity2):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX : <http://example.org/virtualhome2kg/ontology/>
        PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        INSERT { 
            <%activity1> :nextActivity <%activity2> .
            ?firstState :partOf ?situation .
        } WHERE {
            {
                {
                    SELECT (Max(?number) as ?max) WHERE {
                        <%activity1> :action ?action .
                        ?action :actionNumber ?number .
                    }
                }
                <%activity1> :action ?finalAction .
                ?finalAction :actionNumber ?max ;
                        :situationAfterAction ?finalSituation .
                ?finalState :partOf ?finalSituation ;
                            :isStateOf ?object1 .

                <%activity2> :action ?firstAction .
                ?firstAction :actionNumber "0"^^xsd:int;
                    :situationBeforeAction ?firstSituation .
                ?firstState :partOf ?firstSituation ;
                            :isStateOf ?object2 .
                filter(?object1 = ?object2)
            } union {
                {
                    SELECT (Max(?number) as ?max) WHERE {
                        <%activity1> :action ?action .
                        ?action :actionNumber ?number .
                    }
                }
                <%activity1> :action ?finalAction .
                ?finalAction :actionNumber ?max ;
                        :situationAfterAction ?finalSituation .
                ?finalState :partOf ?finalSituation ;
                            :partOf ?situation .
            }
        }
    """
    try :
        sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented/statements")
        sparql.setHTTPAuth(BASIC)
        sparql.setCredentials('admin', '')
        sparql.method = "POST"
        sparql.queryType = "INSERT"
        queryString = queryString.replace("%activity1", activity1)
        queryString = queryString.replace("%activity2", activity2)
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        json = sparql.query().convert()
        bindings = json
        result = bindings
    except  Exception as e:
        print(e.args)
    return result

In [14]:
def preprocess(activity, activityMap, stateMap):
    #get objects' first/final states
    bindings = getFirstAndFinalStates(activity)

    try:
        #preprocess
        for elem in bindings:
            firstState = elem['firstState']['value']
            finalState = elem['finalState']['value']
            #firstStateに関してstateMapの作成
            if 'firstStateType' in elem:
                if firstState not in stateMap:
                    stateMap[firstState] = {
                        "state": [elem['firstStateType']['value']],
                        "object": elem['firstObject']['value']
                    }
                else:
                    if elem['firstStateType']['value'] not in stateMap[firstState]['state']:
                        stateMap[firstState]['state'].append(elem['firstStateType']['value'])
            else:
                if firstState not in stateMap:
                    stateMap[firstState] = {
                        "state": [],
                        "object": elem['firstObject']['value']
                    }
                else:
                    pass

            #finalStateに関してstateMapの作成
            if  'finalStateType' in elem:
                if finalState not in stateMap:
                    stateMap[finalState] = {
                        "state": [elem['finalStateType']['value']],
                        "object": elem['finalObject']['value']
                    }
                else:
                    if elem['finalStateType']['value'] not in stateMap[finalState]['state']:
                        stateMap[finalState]['state'].append(elem['finalStateType']['value'])
            else:
                if finalState not in stateMap:
                    stateMap[finalState] = {
                        "state": [],
                        "object": elem['finalObject']['value']
                    }
                else:
                    pass

            if activity in activityMap:
                if firstState not in activityMap[activity]['firstState']:
                    activityMap[activity]['firstState'].append(firstState)
                if finalState not in activityMap[activity]['finalState']:
                    activityMap[activity]['finalState'].append(finalState)
            else:
                #activity初出時
                activityMap[activity] = {
                    'firstState': [firstState],
                    'finalState': [finalState]
                }
    except Except as e:
        print(e.args)
    return activityMap, stateMap

In [15]:
inverse_states = {
    "http://example.org/virtualhome2kg/ontology/CLOSED": "http://example.org/virtualhome2kg/ontology/OPEN",
    "http://example.org/virtualhome2kg/ontology/OFF": "http://example.org/virtualhome2kg/ontology/ON",
    "http://example.org/virtualhome2kg/ontology/DIRTY": "http://example.org/virtualhome2kg/ontology/CLEAN",
    "http://example.org/virtualhome2kg/ontology/PLUGGED_IN": "http://example.org/virtualhome2kg/ontology/PLUGGED_OUT",
    "http://example.org/virtualhome2kg/ontology/OPEN": "http://example.org/virtualhome2kg/ontology/CLOSED",
    "http://example.org/virtualhome2kg/ontology/ON": "http://example.org/virtualhome2kg/ontology/OFF",
    "http://example.org/virtualhome2kg/ontology/CLEAN": "http://example.org/virtualhome2kg/ontology/DIRTY",
    "http://example.org/virtualhome2kg/ontology/PLUGGED_OUT": "http://example.org/virtualhome2kg/ontology/PLUGGED_IN"
}

In [16]:
routine_list = readRoutine('graph_state_list/routine_list.txt')
executable_routine = 0
for routine in routine_list:
    activity_list = []
    for activity in routine:
        activity = activity.replace(' ', '_')
        activity = activity.lower()
        activity_list.append(getActivity(activity))
    activityMap = {}
    stateMap = {}

    #check consistency
    for i in range(len(activity_list)-1):
        activity1 = activity_list[i]
        activity2 = activity_list[i+1]
        activityMap, stateMap = preprocess(activity1, activityMap, stateMap)
        activityMap, stateMap = preprocess(activity2, activityMap, stateMap)
        consistency = True
        statesChanged = True
        activity1_value = None
        activity2_value = None
        if activity1 == activity2:
            consistency = False
            break
        
        if activity1 in activityMap:
            activity1_value = activityMap[activity1]
            if activity2 in activityMap:
                activity2_value = activityMap[activity2]
            else:
                # if there is no state changed
                statesChanged = False
        else:
            # if there is no state changed
            statesChanged = False

        #print([activity1, activity2])
        try:
            if statesChanged:
                assert checkConsistency(activity1_value, activity2_value)
        except Exception as e:
            consistency = False
            break
        
    #SPARQL DELETE/INSERT query
    if consistency:
        print("Executable")
        print(routine)
        executable_routine += 1
        for i in range(len(activity_list)-1):
            activity1 = activity_list[i]
            activity2 = activity_list[i+1]
#             if checkNextActivityIsRoutined(activity2):
#                     #deleteFinalState(activity1, activity2)
#                     updateRoutine(activity1, activity2)
#                     addNextState(activity1,activity2)
#                     print(str(i) + " updatedRoutine: " + activity1 + " " + activity2)
#             else:
            update(activity1, activity2)
            print(str(i) + " updated: " + activity1 + " " + activity2)
    else:
        print("not executable")

Executable
['Go to sleep', 'Wash teeth', 'Make coffee', 'Have snack', 'Wash dishes', 'Wash dirty dishes', 'Watch youtube', 'Watch movie', 'Wash teeth', 'Keep an eye on stove as something is cooking', 'Have snack', 'Clean', 'Wash teeth', 'Go to sleep', 'Sleep', 'Wash teeth', 'Write an email', 'Take nap']
0 updated: http://example.org/virtualhome2kg/instance/go_to_sleep4_scene1 http://example.org/virtualhome2kg/instance/wash_teeth0_scene1
1 updated: http://example.org/virtualhome2kg/instance/wash_teeth0_scene1 http://example.org/virtualhome2kg/instance/make_coffee0_scene1
2 updated: http://example.org/virtualhome2kg/instance/make_coffee0_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
3 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/wash_dishes4_scene1
4 updated: http://example.org/virtualhome2kg/instance/wash_dishes4_scene1 http://example.org/virtualhome2kg/instance/wash_dirty_dishes0_scene1
5 update

2 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/watch_movie0_scene1
3 updated: http://example.org/virtualhome2kg/instance/watch_movie0_scene1 http://example.org/virtualhome2kg/instance/cook_some_food0_scene1
4 updated: http://example.org/virtualhome2kg/instance/cook_some_food0_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
5 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/wash_dishes4_scene1
6 updated: http://example.org/virtualhome2kg/instance/wash_dishes4_scene1 http://example.org/virtualhome2kg/instance/write_an_email13_scene1
7 updated: http://example.org/virtualhome2kg/instance/write_an_email13_scene1 http://example.org/virtualhome2kg/instance/write_story0_scene1
8 updated: http://example.org/virtualhome2kg/instance/write_story0_scene1 http://example.org/virtualhome2kg/instance/write_an_email15_scene1
9 updated: http://example.

5 updated: http://example.org/virtualhome2kg/instance/cook_some_food5_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
6 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/organize_closet0_scene1
7 updated: http://example.org/virtualhome2kg/instance/organize_closet0_scene1 http://example.org/virtualhome2kg/instance/wash_teeth8_scene1
8 updated: http://example.org/virtualhome2kg/instance/wash_teeth8_scene1 http://example.org/virtualhome2kg/instance/surf_net0_scene1
9 updated: http://example.org/virtualhome2kg/instance/surf_net0_scene1 http://example.org/virtualhome2kg/instance/sleep3_scene1
10 updated: http://example.org/virtualhome2kg/instance/sleep3_scene1 http://example.org/virtualhome2kg/instance/wash_teeth8_scene1
11 updated: http://example.org/virtualhome2kg/instance/wash_teeth8_scene1 http://example.org/virtualhome2kg/instance/wash_sink0_scene1
12 updated: http://example.org/virtualhome2kg/instance

10 updated: http://example.org/virtualhome2kg/instance/relax_on_sofa28_scene1 http://example.org/virtualhome2kg/instance/surf_internet0_scene1
11 updated: http://example.org/virtualhome2kg/instance/surf_internet0_scene1 http://example.org/virtualhome2kg/instance/keep_an_eye_on_stove_as_something_is_cooking0_scene1
12 updated: http://example.org/virtualhome2kg/instance/keep_an_eye_on_stove_as_something_is_cooking0_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
13 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/wash_dishes4_scene1
14 updated: http://example.org/virtualhome2kg/instance/wash_dishes4_scene1 http://example.org/virtualhome2kg/instance/relax_on_sofa28_scene1
15 updated: http://example.org/virtualhome2kg/instance/relax_on_sofa28_scene1 http://example.org/virtualhome2kg/instance/wash_teeth2_scene1
Executable
['Sleep', 'Watch movie', 'Relax on sofa', 'Take nap', 'Breakfast', 'Have snack', 'Wat

not executable
Executable
['Sleep', 'Cook some food', 'Have snack', 'Watch youtube', 'Go to sleep', 'Wash teeth', 'Cut bread', 'Have snack', 'Write an email', 'Breakfast', 'Have snack', 'Write an email', 'Write an email', 'Cook some food', 'Have snack', 'Wash dirty dishes', 'Wash teeth']
0 updated: http://example.org/virtualhome2kg/instance/sleep3_scene1 http://example.org/virtualhome2kg/instance/cook_some_food0_scene1
1 updated: http://example.org/virtualhome2kg/instance/cook_some_food0_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
2 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/watch_youtube0_scene1
3 updated: http://example.org/virtualhome2kg/instance/watch_youtube0_scene1 http://example.org/virtualhome2kg/instance/go_to_sleep5_scene1
4 updated: http://example.org/virtualhome2kg/instance/go_to_sleep5_scene1 http://example.org/virtualhome2kg/instance/wash_teeth0_scene1
5 updated: http://example

...

15 updated: http://example.org/virtualhome2kg/instance/wash_dishes_with_dishwasher13_scene1 http://example.org/virtualhome2kg/instance/surf_internet0_scene1
Executable
['Take nap', 'Make toast', 'Have snack', 'Wash sink', 'Wipe down sink', 'Wash teeth', 'Cook some food', 'Have snack', 'Wash dishes', 'Wash hands', 'Wash dishes', 'Watch movie', 'Cook some food', 'Have snack', 'Wash dishes', 'Watch youtube', 'Breakfast']
0 updated: http://example.org/virtualhome2kg/instance/take_nap0_scene1 http://example.org/virtualhome2kg/instance/make_toast0_scene1
1 updated: http://example.org/virtualhome2kg/instance/make_toast0_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
2 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/wash_sink0_scene1
3 updated: http://example.org/virtualhome2kg/instance/wash_sink0_scene1 http://example.org/virtualhome2kg/instance/wipe_down_sink0_scene1
4 updated: http://example.org/virtualh

8 updated: http://example.org/virtualhome2kg/instance/watch_movie2_scene1 http://example.org/virtualhome2kg/instance/make_toast0_scene1
9 updated: http://example.org/virtualhome2kg/instance/make_toast0_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
10 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/wipe_down_sink0_scene1
11 updated: http://example.org/virtualhome2kg/instance/wipe_down_sink0_scene1 http://example.org/virtualhome2kg/instance/wash_dishes2_scene1
12 updated: http://example.org/virtualhome2kg/instance/wash_dishes2_scene1 http://example.org/virtualhome2kg/instance/write_an_email6_scene1
13 updated: http://example.org/virtualhome2kg/instance/write_an_email6_scene1 http://example.org/virtualhome2kg/instance/keep_an_eye_on_stove_as_something_is_cooking0_scene1
14 updated: http://example.org/virtualhome2kg/instance/keep_an_eye_on_stove_as_something_is_cooking0_scene1 http://example.org/virtua

15 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/wash_dirty_dishes0_scene1
16 updated: http://example.org/virtualhome2kg/instance/wash_dirty_dishes0_scene1 http://example.org/virtualhome2kg/instance/write_an_email14_scene1
Executable
['Take nap', 'Wash dishes', 'Wipe down sink', 'Watch movie', 'Wash teeth', 'Write an email', 'Cook some food', 'Have snack', 'Wash dishes', 'Write an email', 'Write an email', 'Write an email', 'Cut bread', 'Have snack', 'Wash sink', 'Watch movie', 'Watch movie', 'Go to sleep']
0 updated: http://example.org/virtualhome2kg/instance/take_nap0_scene1 http://example.org/virtualhome2kg/instance/wash_dishes0_scene1
1 updated: http://example.org/virtualhome2kg/instance/wash_dishes0_scene1 http://example.org/virtualhome2kg/instance/wipe_down_sink0_scene1
2 updated: http://example.org/virtualhome2kg/instance/wipe_down_sink0_scene1 http://example.org/virtualhome2kg/instance/watch_movie0_scene1
3 upd

1 updated: http://example.org/virtualhome2kg/instance/write_an_email16_scene1 http://example.org/virtualhome2kg/instance/keep_an_eye_on_stove_as_something_is_cooking0_scene1
2 updated: http://example.org/virtualhome2kg/instance/keep_an_eye_on_stove_as_something_is_cooking0_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
3 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/surf_net0_scene1
4 updated: http://example.org/virtualhome2kg/instance/surf_net0_scene1 http://example.org/virtualhome2kg/instance/wash_sink0_scene1
5 updated: http://example.org/virtualhome2kg/instance/wash_sink0_scene1 http://example.org/virtualhome2kg/instance/write_an_email11_scene1
6 updated: http://example.org/virtualhome2kg/instance/write_an_email11_scene1 http://example.org/virtualhome2kg/instance/write_story0_scene1
7 updated: http://example.org/virtualhome2kg/instance/write_story0_scene1 http://example.org/virtualhome2kg/inst

4 updated: http://example.org/virtualhome2kg/instance/organize_closet0_scene1 http://example.org/virtualhome2kg/instance/write_an_email14_scene1
5 updated: http://example.org/virtualhome2kg/instance/write_an_email14_scene1 http://example.org/virtualhome2kg/instance/watch_movie1_scene1
6 updated: http://example.org/virtualhome2kg/instance/watch_movie1_scene1 http://example.org/virtualhome2kg/instance/write_an_email16_scene1
7 updated: http://example.org/virtualhome2kg/instance/write_an_email16_scene1 http://example.org/virtualhome2kg/instance/make_coffee0_scene1
8 updated: http://example.org/virtualhome2kg/instance/make_coffee0_scene1 http://example.org/virtualhome2kg/instance/have_snack0_scene1
9 updated: http://example.org/virtualhome2kg/instance/have_snack0_scene1 http://example.org/virtualhome2kg/instance/wash_monitor0_scene1
10 updated: http://example.org/virtualhome2kg/instance/wash_monitor0_scene1 http://example.org/virtualhome2kg/instance/wash_dishes3_scene1
11 updated: http://e

In [17]:
activityMap

{}

In [18]:
activity_list

['http://example.org/virtualhome2kg/instance/go_to_sleep4_scene1',
 'http://example.org/virtualhome2kg/instance/wash_teeth4_scene1',
 'http://example.org/virtualhome2kg/instance/wash_teeth3_scene1',
 'http://example.org/virtualhome2kg/instance/cut_bread0_scene1',
 'http://example.org/virtualhome2kg/instance/have_snack0_scene1',
 'http://example.org/virtualhome2kg/instance/wash_dishes3_scene1',
 'http://example.org/virtualhome2kg/instance/cook_some_food5_scene1',
 'http://example.org/virtualhome2kg/instance/have_snack0_scene1',
 'http://example.org/virtualhome2kg/instance/watch_movie0_scene1',
 'http://example.org/virtualhome2kg/instance/watch_movie0_scene1',
 'http://example.org/virtualhome2kg/instance/wash_teeth1_scene1',
 'http://example.org/virtualhome2kg/instance/make_coffee0_scene1',
 'http://example.org/virtualhome2kg/instance/have_snack0_scene1',
 'http://example.org/virtualhome2kg/instance/watch_youtube0_scene1',
 'http://example.org/virtualhome2kg/instance/wash_dishes5_scene1'

In [19]:
executable_routine

81